# 06. 統合演習: RAGとLoRAを組み合わせたGradio UIの作成

## 事前準備
Google Colabで **T4 GPU** になっていることを確認してください。

In [ ]:
# 編集禁止セル
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # CUDAデバッグ同期実行

import sys
import glob

import torch
from google.colab import drive

# 1. Google Driveのマウント (05で作成したアダプタの読み込みに必須)
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')

repo_path = '/content/llm_lab'
if os.path.exists(repo_path):
    !rm -rf {repo_path}
!git clone -b stable-base https://github.com/akio-kobayashi/llm_lab.git {repo_path}
os.chdir(repo_path)

!pip install -q -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft datasets gradio
if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

from src.common import load_llm, generate_text
from src.rag import FaissRAGPipeline
from src.ui import create_gradio_ui
from peft import PeftModel
print('セットアップが完了しました。')


## 1. RAGとモデルの準備

05 ノートブックで Google Drive に保存した LoRA アダプタをロードします。

このノートブックでは **再学習は行いません**。`selected_adapter_path.txt` または `lora_runs` 配下の保存済みアダプタを利用します。


In [ ]:
# 編集禁止セル
base_model, tokenizer, rag_pipeline, lora_model = None, None, None, None


def get_adapter_vocab_size(adapter_path: str):
    import os
    import torch
    state = None
    safe_path = os.path.join(adapter_path, 'adapter_model.safetensors')
    bin_path = os.path.join(adapter_path, 'adapter_model.bin')

    try:
        if os.path.exists(safe_path):
            from safetensors.torch import load_file
            state = load_file(safe_path, device='cpu')
        elif os.path.exists(bin_path):
            state = torch.load(bin_path, map_location='cpu')
    except Exception:
        return None

    if state is None:
        return None

    for k in [
        'base_model.model.model.embed_tokens.weight',
        'base_model.model.embed_tokens.weight',
        'base_model.model.lm_head.weight',
        'base_model.lm_head.weight',
    ]:
        if k in state and hasattr(state[k], 'shape') and len(state[k].shape) >= 2:
            return int(state[k].shape[0])

    for k, v in state.items():
        if 'embed_tokens.weight' in k and hasattr(v, 'shape') and len(v.shape) >= 2:
            return int(v.shape[0])

    return None


def normalize_special_tokens_for_inference(tokenizer, model, target_vocab_size=None):
    added = 0

    def can_add_token():
        return target_vocab_size is None or len(tokenizer) < target_vocab_size

    if tokenizer.eos_token is None:
        if can_add_token():
            added += tokenizer.add_special_tokens({'eos_token': '<|eos|>'})
        else:
            tokenizer.eos_token_id = 0
            tokenizer.eos_token = tokenizer.convert_ids_to_tokens(0)

    if tokenizer.bos_token_id == tokenizer.eos_token_id:
        tokenizer.bos_token = None
        tokenizer.bos_token_id = None

    if tokenizer.pad_token is None or tokenizer.pad_token_id == tokenizer.eos_token_id:
        if tokenizer.unk_token is not None and tokenizer.unk_token_id != tokenizer.eos_token_id:
            tokenizer.pad_token = tokenizer.unk_token
            tokenizer.pad_token_id = tokenizer.unk_token_id
        elif can_add_token():
            added += tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
        else:
            fallback_id = 1 if tokenizer.eos_token_id != 1 else 2
            tokenizer.pad_token_id = fallback_id
            tokenizer.pad_token = tokenizer.convert_ids_to_tokens(fallback_id)

    if tokenizer.unk_token is None or tokenizer.unk_token_id == tokenizer.eos_token_id:
        if can_add_token():
            added += tokenizer.add_special_tokens({'unk_token': '<|unk|>'})
        else:
            fallback_id = 1 if tokenizer.eos_token_id != 1 else 2
            tokenizer.unk_token_id = fallback_id
            tokenizer.unk_token = tokenizer.convert_ids_to_tokens(fallback_id)

    if added > 0:
        model.resize_token_embeddings(len(tokenizer))

    print('bos:', tokenizer.bos_token, tokenizer.bos_token_id)
    print('eos:', tokenizer.eos_token, tokenizer.eos_token_id)
    print('pad:', tokenizer.pad_token, tokenizer.pad_token_id)
    print('unk:', tokenizer.unk_token, tokenizer.unk_token_id)

    assert tokenizer.pad_token_id != tokenizer.eos_token_id, 'pad/eos が未分離です'
    assert tokenizer.unk_token_id != tokenizer.eos_token_id, 'unk/eos が未分離です'


def align_model_tokenizer_to_adapter(model, tokenizer, adapter_vocab_size):
    model_vocab = model.get_input_embeddings().weight.shape[0]
    tok_vocab = len(tokenizer)
    print(f'Before align - Tokenizer: {tok_vocab}, Model: {model_vocab}, Adapter: {adapter_vocab_size}')

    normalize_special_tokens_for_inference(tokenizer, model, target_vocab_size=adapter_vocab_size)

    tok_vocab = len(tokenizer)
    model_vocab = model.get_input_embeddings().weight.shape[0]

    if adapter_vocab_size is not None:
        if tok_vocab < adapter_vocab_size:
            extras = [f'<|extra_{i}|>' for i in range(adapter_vocab_size - tok_vocab)]
            tokenizer.add_special_tokens({'additional_special_tokens': extras})
            tok_vocab = len(tokenizer)
            print(f'Tokenizer expanded to: {tok_vocab}')

        if tok_vocab > adapter_vocab_size:
            raise RuntimeError(
                f'Tokenizer vocab ({tok_vocab}) > adapter vocab ({adapter_vocab_size})。'
                '学習時と異なるトークン追加が入っています。'
            )

        if model_vocab != adapter_vocab_size:
            model.resize_token_embeddings(adapter_vocab_size)
            print(f'Model resized to adapter vocab: {adapter_vocab_size}')
    else:
        if model_vocab != tok_vocab:
            model.resize_token_embeddings(tok_vocab)
            print(f'Model resized to tokenizer vocab: {tok_vocab}')

    print(f"After align - eos_id: {tokenizer.eos_token_id}, pad_id: {tokenizer.pad_token_id}")
    print(f"After align - Model vocab: {model.get_input_embeddings().weight.shape[0]}, Tokenizer vocab: {len(tokenizer)}")


try:
    # 1. モデルのロード
    base_model, tokenizer = load_llm(use_4bit=True)

    # 2. RAGの準備 (サンプルデータから即時構築)
    rag_pipeline = FaissRAGPipeline()
    rag_pipeline.build_index('data/docs/anime_docs_sample.jsonl')

    # 3. LoRAアダプタのロード (05で保存した成果物のみ使用)
    base_dir = '/content/drive/MyDrive/llm_lab_outputs'
    selected_path_record = os.path.join(base_dir, 'selected_adapter_path.txt')
    adapter_path = None

    if os.path.exists(selected_path_record):
        with open(selected_path_record, 'r', encoding='utf-8') as f:
            candidate = f.read().strip()
        if candidate and os.path.exists(candidate):
            adapter_path = candidate

    if adapter_path is None:
        candidates = glob.glob('/content/drive/MyDrive/llm_lab_outputs/lora_runs/*/final_adapter')
        candidates = [p for p in candidates if os.path.exists(p)]
        if candidates:
            adapter_path = max(candidates, key=os.path.getmtime)

    if adapter_path is None:
        raise FileNotFoundError(
            'LoRAアダプタが見つかりません。05_lora_concept_demo.ipynb を実行してから再試行してください。'
        )

    adapter_vocab_size = get_adapter_vocab_size(adapter_path)
    print(f'Adapter vocab size: {adapter_vocab_size}')
    align_model_tokenizer_to_adapter(base_model, tokenizer, adapter_vocab_size)

    print(f'LoRAアダプタをロードしています: {adapter_path}')
    lora_model = PeftModel.from_pretrained(base_model, adapter_path)
    lora_model.eval()
    print('すべての準備が整いました。')

except Exception as e:
    print(f'エラーが発生しました: {e}')




## 2. UI起動

入力は任意です。JSON指定・要約指定・自由質問など、複数パターンで動作を確認してください。


In [ ]:
# 編集禁止セル

def safe_generate_local(model, tokenizer, prompt, max_new_tokens=128, repetition_penalty=1.05):
    """06ノート専用: cache互換問題を避けつつ空出力をフォールバックで回避。"""
    try:
        inputs = tokenizer(prompt, return_tensors='pt')
        model_device = next(model.parameters()).device
        inputs = {k: v.to(model_device) for k, v in inputs.items()}
        prompt_len = inputs['input_ids'].shape[1]

        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

        if hasattr(model, 'config'):
            model.config.use_cache = False
        if hasattr(model, 'generation_config'):
            model.generation_config.use_cache = False

        base_kwargs = {
            **inputs,
            'max_new_tokens': max_new_tokens,
            'repetition_penalty': repetition_penalty,
            'pad_token_id': pad_token_id,
            'eos_token_id': tokenizer.eos_token_id,
            'use_cache': False,
        }

        with torch.inference_mode():
            gen_ids = model.generate(**base_kwargs, do_sample=False)

        new_ids = gen_ids[0][prompt_len:]
        text = tokenizer.decode(new_ids, skip_special_tokens=True).strip()

        if not text:
            with torch.inference_mode():
                gen_ids = model.generate(
                    **base_kwargs,
                    do_sample=True,
                    temperature=0.9,
                    top_p=0.95,
                    min_new_tokens=16,
                )
            new_ids = gen_ids[0][prompt_len:]
            text = tokenizer.decode(new_ids, skip_special_tokens=True).strip()

        if not text:
            text = tokenizer.decode(new_ids, skip_special_tokens=False).strip()

        return text
    except Exception as e:
        return f"Error: {e}"


def generate_plain(q):
    return safe_generate_local(base_model, tokenizer, f"### 指示:\n{q}\n\n### 応答:\n").strip()


def generate_rag(q):
    docs = rag_pipeline.search(q, top_k=2)
    prompt = rag_pipeline.create_prompt_with_context(q, docs)
    ans = safe_generate_local(base_model, tokenizer, prompt)
    return ans, "\n\n".join([d['text'] for d in docs])


def generate_lora(q):
    return safe_generate_local(lora_model, tokenizer, f"### 指示:\n{q}\n\n### 応答:\n").strip()


def generate_rag_lora(q):
    docs = rag_pipeline.search(q, top_k=2)
    prompt = rag_pipeline.create_prompt_with_context(q, docs)
    ans = safe_generate_local(lora_model, tokenizer, prompt)
    return ans, "\n\n".join([d['text'] for d in docs])


sample_queries = [
    ["『星屑のメモリー』の主人公について教えて。", True, False],
    ["回答はJSON形式で。『古都の探偵録』のあらすじを2文で教えて。", True, True],
    ["『シャドウ・ハンター』のジャンルを一言で答えて。", False, True],
    ["日本の首都は？理由を一文で。", False, True],
]


demo = create_gradio_ui(generate_plain, generate_rag, generate_lora, generate_rag_lora, examples=sample_queries)
demo.launch(share=True, debug=True)
